In [7]:
!pip install gradio

In [13]:
# import shutil

# def clean_output_folder(output_folder):
#     if os.path.exists(output_folder):
#         shutil.rmtree(output_folder)  # Remove the existing directory and its contents
#     os.makedirs(output_folder)

# clean_output_folder("/kaggle/working/temp_frames")

In [10]:
# def extract_frames(video_path, output_folder="temp_frames", frame_size=(64, 64)):
#     """
#     Extract frames from a single video, resize, and save them to a specified directory.
#     """
#     # Use /kaggle/working/temp_frames to ensure it's a writable directory
#     output_folder = os.path.join("/kaggle/working", output_folder)

#     # Create the directory if it doesn't exist
#     if not os.path.exists(output_folder):
#         os.makedirs(output_folder)
#         print(f"Directory created: {output_folder}")
#     else:
#         print(f"Directory already exists: {output_folder}")

#     if os.access(output_folder, os.W_OK):
#         print(f"Directory is writable: {output_folder}")
#     else:
#         print(f"Directory is NOT writable: {output_folder}")

    
#     video_capture = cv2.VideoCapture(video_path)
#     frame_count = 0
#     success = True
#     extracted_frames = []

#     while success:
#         success, frame = video_capture.read()
#         if success:
#             # Resize and convert to grayscale
#             frame = cv2.resize(frame, frame_size, interpolation=cv2.INTER_AREA)
#             frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

#             # Save frame
#             frame_filename = f"frame_{frame_count:05d}.jpg"
#             frame_filepath = os.path.join(output_folder, frame_filename)
#             cv2.imwrite(frame_filepath, frame)

#             extracted_frames.append(frame_filepath)
#             frame_count += 1

#     video_capture.release()
#     return extracted_frames

In [12]:
# video_path = "/kaggle/input/ucf101-action-recognition/test/BenchPress/v_BenchPress_g01_c02.avi"  # Update this to a valid video path
# frames = extract_frames(video_path, output_folder="temp_frames")
# print(f"Extracted {len(frames)} frames to temp_frames.")

Extracted 180 frames to temp_frames.


In [15]:
import gradio as gr
import cv2
import os

# Function to extract frames
def extract_frames(video_path, output_folder="/kaggle/working/temp_frames", frame_size=(64, 64)):
    """
    Extract frames from a video, resize to 64x64, and save to the specified folder.

    Parameters:
        video_path (str): Path to the video file.
        output_folder (str): Directory to save extracted frames.
        frame_size (tuple): Target frame size (width, height).

    Returns:
        list: List of file paths to the extracted frames.
    """
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
        print(f"Directory created: {output_folder}")
    else:
        print(f"Directory already exists: {output_folder}")

    video_capture = cv2.VideoCapture(video_path)
    frame_count = 0
    success = True
    extracted_frames = []

    while success:
        success, frame = video_capture.read()
        if success:
            # Resize and convert to grayscale
            frame = cv2.resize(frame, frame_size, interpolation=cv2.INTER_AREA)
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

            # Save frame as image
            frame_filename = f"frame_{frame_count:05d}.jpg"
            frame_filepath = os.path.join(output_folder, frame_filename)
            cv2.imwrite(frame_filepath, frame)

            extracted_frames.append(frame_filepath)
            frame_count += 1

    video_capture.release()
    print(f"Extracted {len(extracted_frames)} frames.")
    return extracted_frames


# Function to process video input
def process_video(uploaded_video, selected_sample):
    """
    Process the uploaded or selected video and extract frames.

    Parameters:
        uploaded_video (Gradio file): Uploaded video by user.
        selected_sample (str): Path to a selected video from test dataset.

    Returns:
        list: List of paths to extracted frames.
    """
    if uploaded_video is not None:
        # Save uploaded video to /kaggle/working/temp_videos
        temp_dir = "/kaggle/working/temp_videos"
        os.makedirs(temp_dir, exist_ok=True)
        video_path = os.path.join(temp_dir, uploaded_video.name)
        with open(video_path, "wb") as f:
            f.write(uploaded_video.read())
        print(f"Uploaded video saved at: {video_path}")
    elif selected_sample is not None:
        # Use selected sample video path
        video_path = selected_sample
        print(f"Selected sample video path: {video_path}")
    else:
        print("No video input provided.")
        return []

    # Extract frames
    extracted_frames = extract_frames(video_path, output_folder="/kaggle/working/temp_frames")
    return extracted_frames


# Video paths for Dropdown (example paths)
video_paths = [
    "/kaggle/input/ucf101-action-recognition/test/JumpingJack/k_JumpingJack_g12_c04.avi",
    "/kaggle/input/ucf101-action-recognition/test/Walking/k_Walking_g12_c04.avi",
    "/kaggle/input/ucf101-action-recognition/test/Biking/k_Biking_g12_c04.avi",
]

# Gradio Interface
with gr.Blocks() as demo:
    gr.Markdown("# Video Processing Interface")
    gr.Markdown("Upload a video or select a video from the test folder to process and extract frames.")

    with gr.Row():
        with gr.Column():
            # Upload option
            video_input = gr.File(label="Upload Video", file_types=[".mp4", ".avi", ".mkv"])
            # Dropdown selection
            sample_input = gr.Dropdown(
                choices=video_paths,
                label="Or Select a Video from Test Folder",
                interactive=True
            )
            run_button = gr.Button("Run Models")
        with gr.Column():
            with gr.Tab("Model 1"):
                gr.Markdown("### Model 1 Results")
                frames_output1 = gr.Gallery(label="Extracted Frames", columns=5, height="auto")  # For testing
                video_output1 = gr.Video(label="Final Video")  # Placeholder
                time_output1 = gr.Textbox(label="Inference Time")  # Placeholder
            with gr.Tab("Model 2"):
                gr.Markdown("### Model 2 Results")
                frames_output2 = gr.Gallery(label="Generated Frames", columns=2, height="auto")
                video_output2 = gr.Video(label="Final Video")
                time_output2 = gr.Textbox(label="Inference Time")
            with gr.Tab("Model 3"):
                gr.Markdown("### Model 3 Results")
                frames_output3 = gr.Gallery(label="Generated Frames", columns=2, height="auto")
                video_output3 = gr.Video(label="Final Video")
                time_output3 = gr.Textbox(label="Inference Time")

    # Handle button click
    def handle_run(uploaded_video, selected_sample):
        frames = process_video(uploaded_video, selected_sample)
        return frames, None, "0.0 seconds", [], None, None, [], None, None

    run_button.click(
        handle_run,
        inputs=[video_input, sample_input],
        outputs=[
            frames_output1, video_output1, time_output1,
            frames_output2, video_output2, time_output2,
            frames_output3, video_output3, time_output3
        ]
    )

    gr.Markdown("© 2024 Your Application")

# Launch the application
demo.launch()

* Running on local URL:  http://127.0.0.1:7864
Kaggle notebooks require sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

* Running on public URL: https://00921f2dd38ee6c809.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Selected sample video path: /kaggle/input/ucf101-action-recognition/test/JumpingJack/k_JumpingJack_g12_c04.avi
Directory already exists: /kaggle/working/temp_frames
Extracted 0 frames.
